In [36]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv1D, LeakyReLU, BatchNormalization, Conv1DTranspose, Activation, GRU, ELU, ReLU, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, regularizers

In [11]:
BATCH_SIZE = 64
NOISE_DIM = 100
EPOCHS = 100
TRAIN_RATIO = 5  # 训练critic的次数
CLIP_VALUE = 0.01  # 用于权重剪裁
LEARNING_RATE = 0.0001

In [4]:
# Fetch BTC trade raw data
today = datetime.date.today()
BTC_raw = yf.download('BTC-USD', start=today-datetime.timedelta(days=700), end=today)
# BTC_raw.columns = ['date', 'open_price', 'high', 'low', 'close_price', 'adj close', 'volume']
BTC_raw.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-05-28,38507.082031,38856.968750,34779.039062,35697.605469,35697.605469,55200191952
2021-05-29,35684.156250,37234.500000,33693.929688,34616.066406,34616.066406,45231013335
2021-05-30,34607.406250,36400.667969,33520.738281,35678.128906,35678.128906,31646080921
2021-05-31,35658.593750,37468.250000,34241.945312,37332.855469,37332.855469,39009847639
2021-06-01,37293.792969,37896.734375,35787.085938,36684.925781,36684.925781,34639423297


In [16]:
# Split the raw data into parts
BTC_basic = BTC_raw[['Adj Close', 'Open']]
BTC_quant = BTC_raw[['Open', 'High', 'Low', 'Close', 'Volume']].values
BTC_quant

array([[3.85070820e+04, 3.88569688e+04, 3.47790391e+04, 3.56976055e+04,
        5.52001920e+10],
       [3.56841562e+04, 3.72345000e+04, 3.36939297e+04, 3.46160664e+04,
        4.52310133e+10],
       [3.46074062e+04, 3.64006680e+04, 3.35207383e+04, 3.56781289e+04,
        3.16460809e+10],
       ...,
       [2.75917305e+04, 2.79799824e+04, 2.70708496e+04, 2.75253398e+04,
        1.77032883e+10],
       [2.75148730e+04, 2.83710781e+04, 2.72079316e+04, 2.83075977e+04,
        1.77333731e+10],
       [2.83000586e+04, 2.99958379e+04, 2.73245488e+04, 2.84227012e+04,
        3.18542420e+10]])

In [34]:
# WGAN-GP models
def build_generator(input_dim):
    input_layer = Input(shape=(input_dim,))
    x = Dense(128)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(input_dim, activation='tanh')(x)
    model = Model(input_layer, x)
    return model

def build_critic():
    model_input = Input(shape=(5,))
    model = Dense(320)(model_input)
    model = Reshape((1, 5, 64))(model)
    model = Conv2D(32, kernel_size=(1, 3), padding='same')(model)
    model = LeakyReLU(alpha=0.2)(model)
    model = Conv2D(64, kernel_size=(1, 3), padding='same')(model)
    model = LeakyReLU(alpha=0.2)(model)
    model = Flatten()(model)
    model_output = Dense(1)(model)
    return Model(model_input, model_output)

In [37]:
generator = build_generator(NOISE_DIM)
critic = build_critic()

critic_optimizer = Adam(learning_rate=LEARNING_RATE, beta_1=0.5, beta_2=0.9)
critic.compile(loss='mse', optimizer=critic_optimizer)

In [47]:
def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)

def gradient_penalty_loss(y_true, y_pred, interpolated_samples):
    gradients = tf.gradients(y_pred, interpolated_samples)[0]
    gradients_sqr = tf.square(gradients)
    gradients_sqr_sum = tf.reduce_sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = tf.sqrt(gradients_sqr_sum)
    gradient_penalty = tf.reduce_mean((gradient_l2_norm - 1.)**2)
    return gradient_penalty

def train_wgan_gp(stock_data, generator, critic, batch_size, noise_dim, epochs, train_ratio, clip_value, learning_rate):
    num_samples = stock_data.shape[0]

    real_stock_data = Input(shape=(stock_data.shape[1],))
    noise_input = Input(shape=(noise_dim,))
    
    generated_stock_data = generator(noise_input)

    real_stock_score = critic(real_stock_data)
    generated_stock_score = critic(generated_stock_data)

    alpha = tf.random.uniform(shape=[batch_size, 1], minval=0., maxval=1.)
    interpolated = (alpha * real_stock_data) + ((1 - alpha) * generated_stock_data)
    gp = gradient_penalty_loss(real_stock_data, generated_stock_score, interpolated)

    wasserstein_distance = real_stock_score - generated_stock_score
    critic_loss = wasserstein_loss(real_stock_data, generated_stock_score) + 10 * gp

    critic_optimizer = Adam(lr=learning_rate, beta_1=0.5, beta_2=0.9)
    critic_model = Model(inputs=[real_stock_data, noise_input], outputs=[wasserstein_distance, gp])
    #critic_model.compile(loss=[wasserstein_loss, gradient_penalty_loss], optimizer=critic_optimizer, loss_weights=[1, 10])
    critic.compile(optimizer=Adam(lr=learning_rate), loss=wasserstein_loss, metrics=['accuracy'])

    generator_score = critic(generated_stock_data)
    generator_model = Model(inputs=noise_input, outputs=generator_score)
    generator_model.compile(loss=wasserstein_loss, optimizer=critic_optimizer)

    # 训练循环
    for epoch in range(epochs):
        print("Epoch", epoch)
        for i in range(0, num_samples, batch_size):
            for _ in range(train_ratio):
                real_stock_batch = stock_data[np.random.randint(0, num_samples, size=batch_size)]
                noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
                critic_loss = critic.train_on_batch([real_stock_batch, generator.predict(noise)], [-np.ones((batch_size, 1)), np.ones((batch_size, 1))])
                for layer in critic.layers:
                    weights = layer.get_weights()
                    weights = [np.clip(w, -clip_value, clip_value) for w in weights]
                    layer.set_weights(weights)

            noise = np.random.normal(0, 1, size=(batch_size, 5))
            generated_data = generator.predict(noise)
            generator_loss = generator_model.train_on_batch(generated_data, np.ones((batch_size, 1)))

        print("Critic Loss:", critic_loss)
        print("Generator Loss:", generator_loss)

        if epoch % 10 == 0:
            print("Generating sample data...")
            generated_data = generator.predict(np.random.normal(0, 1, size=(10, noise_dim)))
            print("Sample Data:", generated_data)

    return generator


In [48]:
trained_generator = train_wgan_gp(BTC_quant, generator, critic, BATCH_SIZE, NOISE_DIM, EPOCHS, TRAIN_RATIO, CLIP_VALUE, LEARNING_RATE)

ValueError: ignored